In [1]:
import pandas as pd
import geopandas as gpd
import time
import itertools
import numpy as np
import folium
from scipy.spatial import KDTree, ConvexHull
import networkx as nx
from shapely.geometry import Polygon
import folium

In [2]:
data = pd.read_csv("./property_locations.csv")
data['addr_date'] = pd.to_datetime(data['addr_date'])
# data = data.set_index('UDPRN')
data

,UDPRN,UPRN,POSTCODE,STREET_DESCRIPTION,x,y,lat,lng,addr_date
0,10314712,10090973062,GU31 4AW,READON CLOSE,475167.46875,123746.570312,51.008198,-0.929962,2012-03-19
1,10314713,10090973064,GU31 4AW,READON CLOSE,475167.46875,123746.570312,51.008198,-0.929962,2012-03-19
2,10314714,10090973065,GU31 4AW,READON CLOSE,475167.46875,123746.570312,51.008198,-0.929962,2012-03-19
3,10314715,10090973066,GU31 4AW,READON CLOSE,475167.46875,123746.570312,51.008198,-0.929962,2012-03-19
4,10314716,10090973068,GU31 4AW,READON CLOSE,475167.46875,123746.570312,51.008198,-0.929962,2012-03-19
...,...,...,...,...,...,...,...,...,...
509,56694878,10096346855,GU31 4GF,DRAGON STREET,474810.00000,123181.218750,51.003159,-0.935172,2022-06-17
510,56694879,10096346856,GU31 4GF,DRAGON STREET,474810.00000,123181.218750,51.003159,-0.935172,2022-06-17
511,56694880,10096346857,GU31 4GF,DRAGON STREET,474810.00000,123181.218750,51.003159,-0.935172,2022-06-17
512,56694881,10096346858,GU31 4GF,DRAGON STREET,474810.00000,123181.218750,51.003159,-0.935172,2022-06-17


In [31]:
def find_clusters(data, x_col, y_col, max_distance, min_cluster_size=15, date_col=None, max_date_difference=None, return_data=True):
    kdtree = KDTree(data[[x_col, y_col]])
    pairs = np.array(list(kdtree.query_pairs(r=max_distance)))

    # Create a network graph to convert pairings into clusters
    g = nx.Graph()
    g.add_nodes_from(data.index)
    g.add_edges_from(pairs)
    nx.set_node_attributes(g, data.to_dict('index'))

    # If a date column is specified, calculate the date difference.
    # Add this as an edge attribute
    if date_col is not None:
        for u, v in g.edges():
            d = abs((g.nodes[u][date_col] - g.nodes[v][date_col]).days)/365
            g[u][v]['dt'] = d

    # If max date difference is specificed,
    # loop each edge and remove those that are below this.
    if max_date_difference is not None:
        for u, v in g.edges():
            d = g[u][v]['dt']
            if d > max_date_difference:
                g.remove_edge(u, v)

    # With NX, we can easily find connected components, removing those nodes that are not part of any cluster
    components = [g.subgraph(c).copy() for c in nx.connected_components(g)]

    # We also want to keep clusters of at least 15 properties.
    clusters = [i for i in components if len(i.nodes()) >= min_cluster_size]

    # Assign cluster IDs to each cluster
    cluster_IDs = {idx: np.array(c.nodes()) for (idx, c) in enumerate(clusters, start=1)}
    attr = {n: i for (i, nodes) in cluster_IDs.items() for n in nodes} # Reverse so its node to ID

    # First create a new node attribute and set to NaN. Then fill with cluster IDs on clustered nodes
    nx.set_node_attributes(g, np.nan, name='cluster_ID')
    nx.set_node_attributes(g, attr, name='cluster_ID')

    # Now we can easily recreate the dataframe from all the nodes.
    # This will now include a new column for cluster ID.
    df_data = pd.DataFrame.from_dict(dict(g.nodes(data=True)), orient='index')

    # Create the convex hull polygons
    geoms = []
    for i in cluster_IDs.keys():
        points = df_data[df_data['cluster_ID']==i][[x_col, y_col]].values
        try:
            hull = ConvexHull(points)
            hull = list(zip(points[hull.vertices, 0], points[hull.vertices, 1]))
            geom = Polygon(hull)
        except:
            geom = np.nan

        geoms.append({
            'cluster_ID': i,
            'geometry': geom
        })

    df_geoms = pd.DataFrame(geoms)

    if return_data:
        return df_geoms, df_data

    return df_geoms

In [35]:
df_geoms, df_data = find_clusters(data, 'x', 'y', 50, min_cluster_size=15, date_col='addr_date', max_date_difference=0.5, return_data=True)
df_geoms


,cluster_ID,geometry
0,1,NaN
1,2,"POLYGON ((474326.21875 122444.3515625, 474225...."
2,3,"POLYGON ((473886.46875 122510.5703125, 473859...."
3,4,"POLYGON ((474468.3125 122495.296875, 474464.15..."


In [29]:
df_data

,UDPRN,UPRN,POSTCODE,STREET_DESCRIPTION,x,y,lat,lng,addr_date,cluster_ID
0,10314712,10090973062,GU31 4AW,READON CLOSE,475167.46875,123746.570312,51.008198,-0.929962,2012-03-19,NaN
1,10314713,10090973064,GU31 4AW,READON CLOSE,475167.46875,123746.570312,51.008198,-0.929962,2012-03-19,NaN
2,10314714,10090973065,GU31 4AW,READON CLOSE,475167.46875,123746.570312,51.008198,-0.929962,2012-03-19,NaN
3,10314715,10090973066,GU31 4AW,READON CLOSE,475167.46875,123746.570312,51.008198,-0.929962,2012-03-19,NaN
4,10314716,10090973068,GU31 4AW,READON CLOSE,475167.46875,123746.570312,51.008198,-0.929962,2012-03-19,NaN
...,...,...,...,...,...,...,...,...,...,...
509,56694878,10096346855,GU31 4GF,DRAGON STREET,474810.00000,123181.218750,51.003159,-0.935172,2022-06-17,NaN
510,56694879,10096346856,GU31 4GF,DRAGON STREET,474810.00000,123181.218750,51.003159,-0.935172,2022-06-17,NaN
511,56694880,10096346857,GU31 4GF,DRAGON STREET,474810.00000,123181.218750,51.003159,-0.935172,2022-06-17,NaN
512,56694881,10096346858,GU31 4GF,DRAGON STREET,474810.00000,123181.218750,51.003159,-0.935172,2022-06-17,NaN


In [30]:
data

,UDPRN,UPRN,POSTCODE,STREET_DESCRIPTION,x,y,lat,lng,addr_date
0,10314712,10090973062,GU31 4AW,READON CLOSE,475167.46875,123746.570312,51.008198,-0.929962,2012-03-19
1,10314713,10090973064,GU31 4AW,READON CLOSE,475167.46875,123746.570312,51.008198,-0.929962,2012-03-19
2,10314714,10090973065,GU31 4AW,READON CLOSE,475167.46875,123746.570312,51.008198,-0.929962,2012-03-19
3,10314715,10090973066,GU31 4AW,READON CLOSE,475167.46875,123746.570312,51.008198,-0.929962,2012-03-19
4,10314716,10090973068,GU31 4AW,READON CLOSE,475167.46875,123746.570312,51.008198,-0.929962,2012-03-19
...,...,...,...,...,...,...,...,...,...
509,56694878,10096346855,GU31 4GF,DRAGON STREET,474810.00000,123181.218750,51.003159,-0.935172,2022-06-17
510,56694879,10096346856,GU31 4GF,DRAGON STREET,474810.00000,123181.218750,51.003159,-0.935172,2022-06-17
511,56694880,10096346857,GU31 4GF,DRAGON STREET,474810.00000,123181.218750,51.003159,-0.935172,2022-06-17
512,56694881,10096346858,GU31 4GF,DRAGON STREET,474810.00000,123181.218750,51.003159,-0.935172,2022-06-17


In [25]:
df_geoms, df_data = find_clusters(data, 'x', 'y', 50, min_cluster_size=15, date_col='addr_date', max_date_difference=0.5, return_data=True)

df_geoms = gpd.GeoDataFrame(df_geoms, geometry='geometry', crs='EPSG:27700')

m = folium.Map(zoom_start=8, tiles='cartodbpositron')

folium.GeoJson(
    data=df_geoms.dropna(),
    popup=folium.GeoJsonPopup(fields=['cluster_ID']),
    style_function=lambda x: {'color': 'blue'}
    ).add_to(m)

df = df_data.dropna(subset=['cluster_ID'])[['UDPRN', 'lat', 'lng']]
for idx, row in df.iterrows():
    folium.CircleMarker(
        location=(row['lat'], row['lng']),
        radius=1,
        fill=True,
        color='grey',
        fill_color='grey',
        popup=str(row[['UDPRN']].to_dict())
    ).add_to(m)

m.fit_bounds(m.get_bounds())
m

In [ ]:
kdtree = KDTree(data[['x', 'y']])
pairs = np.array(list(kdtree.query_pairs(r=50)))

g = nx.Graph()
g.add_nodes_from(data.index)
nx.set_node_attributes(g, data.to_dict('index'))

# Create a network graph to convert pairings into clusters
g.add_edges_from(pairs)

for u, v in g.edges():
    d = abs((g.nodes[u]['addr_date'] - g.nodes[v]['addr_date']).days)/365
    g[u][v]['dt'] = d

for u, v in g.edges():
    d = g[u][v]['dt']
    if d>0.5:
        g.remove_edge(u, v)

# With NX, we can easily find connected components, removing those nodes that are not part of any cluster
components = [g.subgraph(c).copy() for c in nx.connected_components(g)]

# We also want to keep clusters of at least 15 properties.
clusters = [i for i in components if len(i.nodes())>=15]

cluster_IDs = {idx: np.array(c.nodes()) for (idx, c) in enumerate(clusters, start=1) if len(c.nodes())>=15}
attr = {n: i for (i, nodes) in cluster_IDs.items() for n in nodes}

nx.set_node_attributes(g, np.nan, name='cluster_ID')
nx.set_node_attributes(g, attr, name='cluster_ID')

geoms = []
for i in cluster_IDs.keys():
    points = df_data[df_data['cluster_ID']==i][['x', 'y']].values
    try:
        hull = ConvexHull(points)
        hull = list(zip(points[hull.vertices, 0], points[hull.vertices, 1]))
        geom = Polygon(hull)
    except:
        geom = np.nan

    geoms.append({
        'cluster_ID': i,
        'geometry': geom
    })


df_data = pd.DataFrame.from_dict(dict(g.nodes(data=True)), orient='index')
df_geoms = pd.DataFrame(geoms)
df_geoms


In [ ]:
hulls = []
all_properties = []
for idx, c in enumerate(clusters, start=1):
    points = np.array(list(zip(
        nx.get_node_attributes(c, 'x').values(),
        nx.get_node_attributes(c, 'y').values())
        ))
    dates = list(nx.get_node_attributes(c, 'addr_date').values())

    try:
        hull = ConvexHull(points)
        hull = list(zip(points[hull.vertices, 0], points[hull.vertices, 1]))

        hulls.append({
            "nr": idx,
            "nr_props": len(points),
            "date_from": min(dates),
            "date_to": max(dates),
            "geometry": Polygon(hull),
            })

        for n in c.nodes():
            all_properties.append({'UDPRN': n, 'ClusterNr': idx})

    except:
        pass

hulls = gpd.GeoDataFrame(hulls, geometry='geometry', crs='EPSG:27700')
all_properties = pd.DataFrame(all_properties)

print(f"Nr clusters found: {len(clusters)}")
print(f"Nr of convex hulls found: {len(hulls)}")
print(f"Nr of properties in hulls: {len(all_properties)}")

hulls

In [ ]:
# data = data.reset_index(drop=False)

# ====== Find clusters of properties ======
# Find property pairs within a given distance of each other
# AND where their address dates are within 6 months of each other.
kdtree = KDTree(data[['x', 'y']])
pairs = kdtree.query_pairs(r=50)

# Get the source and target ie the closest matching pair
results = []
for (i, j) in pairs:
    i_id = data.loc[i, 'UDPRN']
    j_id = data.loc[j, 'UDPRN']
    i_date = data.loc[i, 'addr_date']
    j_date = data.loc[j, 'addr_date']
    results.append((i_id, j_id, i_date, j_date))

results = pd.DataFrame(results, columns=['source', 'target', 'source_date', 'target_date'])
results['dt'] = (abs((results['target_date'] - results['source_date']).dt.days)/365).round(2)
results = results[results['dt']<=0.5]

results



In [ ]:
# Now we have pairs, we can create a NetworkX graph and add property attributes

data = data.set_index('UDPRN')
all_nodes = set(list(results['source'].unique()) + list(results['target'].unique()))

g = nx.Graph()
g.add_nodes_from(all_nodes)
nx.set_node_attributes(g, data.loc[all_nodes].to_dict('index'))

# Create a network graph to convert pairings into clusters
g.add_edges_from(results[['source', 'target']].values)


In [ ]:
# With NX, we can easily find connected components, removing those nodes that are not part of any cluster
components = [g.subgraph(c).copy() for c in nx.connected_components(g)]
print(len(components))

# We also want to keep clusters of at least 15 properties.
clusters = [i for i in components if len(i.nodes())>=15]
print(len(clusters))

In [ ]:
hulls = []
all_properties = []
for idx, c in enumerate(clusters, start=1):
    points = np.array(list(zip(
        nx.get_node_attributes(c, 'x').values(),
        nx.get_node_attributes(c, 'y').values())
        ))
    dates = list(nx.get_node_attributes(c, 'addr_date').values())

    try:
        hull = ConvexHull(points)
        hull = list(zip(points[hull.vertices, 0], points[hull.vertices, 1]))

        hulls.append({
            "nr": idx,
            "nr_props": len(points),
            "date_from": min(dates),
            "date_to": max(dates),
            "geometry": Polygon(hull),
            })

        for n in c.nodes():
            all_properties.append({'UDPRN': n, 'ClusterNr': idx})

    except:
        pass

hulls = gpd.GeoDataFrame(hulls, geometry='geometry', crs='EPSG:27700')
all_properties = pd.DataFrame(all_properties)

print(f"Nr clusters found: {len(clusters)}")
print(f"Nr of convex hulls found: {len(hulls)}")
print(f"Nr of properties in hulls: {len(all_properties)}")

hulls